# Project Setup

This notebook will guide you through setting up the project environment for using CrewAI. We will:

1. Install the required Python modules.
2. Set up a virtual environment.
3. Verify the installation.

In [ ]:
# Uncomment if you are not using devcontainers and want to set up a local environment
# 
# # Step 1: Create and activate a virtual environment
# #
# %python3 -m venv venv
# %source venv/bin/activate
# 
# # Step 2: Install required Python modules
# #
# %pip install -r requirements.txt
# 
# # Step 3: Verify installation
# #
# %pip list

# Load Required Python Modules and Libraries

In [1]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown
from crewai import LLM, Agent, Task, Crew

In [2]:
# Uncomment in order to enable litellm debugging for better error diagnostics
#
import litellm
litellm._turn_on_debug()
print("✅ LiteLLM debugging enabled")

# Load Environment Variables and Configure LLM

This block loads environment variables from the `.env` file, including the OpenAI API Key, which is required to authenticate with OpenAI's services. It then configures the `LLM` object to use OpenAI's GPT-4 model. Alternatively, you can uncomment the provided code to configure the `LLM` object to use Ollama with a local model, provided Ollama is installed and running.

In [3]:
# Load environment variables from .env file
# Note: In devcontainer, variables are already loaded by dotenv feature,
# but load_dotenv() is safe and won't override existing environment variables
load_dotenv()

# Uncomment the code block below to use OpenAI with your API Key
# 
# api_key = os.getenv('OPENAI_API_KEY')
# if not api_key:
#     raise ValueError("OPENAI_API_KEY is not set in the .env file")

# Uncomment the code block below to use ollama
# 
OLLAMA_API_BASE = os.getenv('OLLAMA_API_BASE')

# Configure LLM

Configures the `LLM` object to use OpenAI's GPT-4 model. 

Alternatively, you can uncomment the provided code to configure the `LLM` object to use Ollama with a local model, provided Ollama is installed and running.

In [4]:
# llm = LLM(
#     model="gpt-4o",  # Specify the OpenAI model you want to use
#     api_key=api_key
# )

# Uncomment the code block below to use Ollama with your local model
# Make sure to have Ollama installed and running
# 
llm = LLM(
    model="ollama/gemma3n:latest",
    base_url=OLLAMA_API_BASE
)

In [5]:
# Uncomment to test ollam connectivity
# 
import requests

# Debug: Check environment variables and test Ollama connection
# 
print("=== Environment Variables Debug ===")
print(f"OLLAMA_API_BASE: {OLLAMA_API_BASE}")
test_url = f"{OLLAMA_API_BASE}/api/tags"

# Test connection to your Ollama server
# 
try:
    response = requests.get(test_url, timeout=5)
    print(f"\n=== Ollama Connection Test ===")
    print(f"URL: {test_url}")
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
except Exception as e:
    print(f"Connection failed: {e}")

# Debug: Check what base_url is actually being used
# 
print(f"\n=== CrewAI LLM Connection Test ===")
# print(f"ollama_base_url from env: {OLLAMA_API_BASE}")
print(f"LLM base_url: {llm.base_url}")
print(f"LLM model: {llm.model}")

# Test a simple LLM call
try:
    test_response = llm.call([{"role": "user", "content": "Say hello!"}])
    print("✅ LLM test successful!")
except Exception as e:
    print(f"❌ LLM test failed: {e}")

20:00:33 - LiteLLM:DEBUG: utils.py:338 - 

20:00:33 - LiteLLM:DEBUG: utils.py:338 - Request to litellm:
20:00:33 - LiteLLM:DEBUG: utils.py:338 - litellm.completion(model='ollama/gemma3n:latest', messages=[{'role': 'user', 'content': 'Say hello!'}], stop=[], base_url='http://host.docker.internal:11434', stream=False)
20:00:33 - LiteLLM:DEBUG: utils.py:338 - 

20:00:33 - LiteLLM:DEBUG: litellm_logging.py:460 - self.optional_params: {}
20:00:33 - LiteLLM:DEBUG: utils.py:338 - SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
20:00:33 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= gemma3n:latest; provider = ollama
20:00:33 - LiteLLM:DEBUG: utils.py:338 - Request to litellm:
20:00:33 - LiteLLM:DEBUG: utils.py:338 - litellm.completion(model='ollama/gemma3n:latest', messages=[{'role': 'user', 'content': 'Say hello!'}], stop=[], base_url='http://host.docker.internal:11434', stream=False)
20:00:33 - LiteLLM:DEBUG: utils.py:338 - 

20:00:33 - 

=== Environment Variables Debug ===
OLLAMA_API_BASE: http://host.docker.internal:11434

=== Ollama Connection Test ===
URL: http://host.docker.internal:11434/api/tags
Status: 200
Response: {'models': [{'name': 'gemma3n:latest', 'model': 'gemma3n:latest', 'modified_at': '2025-06-30T14:40:46.504797444-04:00', 'size': 7547589116, 'digest': '15cb39fd9394fd2549f6df9081cfc84dd134ecf2c9c5be911e5629920489ac32', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'gemma3n', 'families': ['gemma3n'], 'parameter_size': '6.9B', 'quantization_level': 'Q4_K_M'}}, {'name': 'llama3:latest', 'model': 'llama3:latest', 'modified_at': '2025-06-29T18:13:52.360077836-04:00', 'size': 4661224676, 'digest': '365c0bd3c000a25d28ddbf732fe1c6add414de7275464c4e4d1c3b5fcb5d8ad1', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'llama', 'families': ['llama'], 'parameter_size': '8.0B', 'quantization_level': 'Q4_0'}}]}

=== CrewAI LLM Connection Test ===
LLM base_url: http://host.docker.internal:1143

20:00:34 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
20:00:34 - LiteLLM:DEBUG: litellm_logging.py:1352 - Logging Details LiteLLM-Success Call: Cache_hit=None
20:00:34 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: ollama/gemma3n:latest
20:00:34 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: ollama/gemma3n:latest
20:00:34 - LiteLLM:DEBUG: utils.py:4349 - checking potential_model_names in litellm.model_cost: {'split_model': 'gemma3n:latest', 'combined_model_name': 'ollama/gemma3n:latest', 'stripped_model_name': 'gemma3n:latest', 'combined_stripped_model_name': 'ollama/gemma3n:latest', 'custom_llm_provider': 'ollama'}
20:00:34 - LiteLLM:DEBUG: utils.py:4349 - checking potential_model_names in litellm.model_cost: {'split_model': 'gemma3n:latest', 'combined_model_name': 'ollama/gemma3n:latest', 'stripped_model_name': 'gemma3n:latest', 'combined_stripped_model_name': 'ollama/gemma3n:l

✅ LLM test successful!


20:00:34 - LiteLLM:DEBUG: utils.py:4349 - checking potential_model_names in litellm.model_cost: {'split_model': 'gemma3n:latest', 'combined_model_name': 'ollama/gemma3n:latest', 'stripped_model_name': 'gemma3n:latest', 'combined_stripped_model_name': 'ollama/gemma3n:latest', 'custom_llm_provider': 'ollama'}


20:00:34 - LiteLLM:DEBUG: utils.py:4649 - model_info: {'key': 'gemma3n:latest', 'litellm_provider': 'ollama', 'mode': 'chat', 'supports_function_calling': False, 'input_cost_per_token': 0.0, 'output_cost_per_token': 0.0, 'max_tokens': 32768, 'max_input_tokens': 32768, 'max_output_tokens': 32768}
20:00:34 - LiteLLM:DEBUG: litellm_logging.py:1377 - Logging Details LiteLLM-Success Call streaming complete
20:00:34 - LiteLLM:DEBUG: litellm_logging.py:1377 - Logging Details LiteLLM-Success Call streaming complete
20:00:34 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: ollama/gemma3n:latest
20:00:34 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: ollama/gemma3n:latest
20:00:34 - LiteLLM:DEBUG: utils.py:4349 - checking potential_model_names in litellm.model_cost: {'split_model': 'gemma3n:latest', 'combined_model_name': 'ollama/gemma3n:latest', 'stripped_model_name': 'gemma3n:latest', 'combined_stripped_model_name': 'ollama/

# Single Agent Single Tool Example

## Define Agents

This block defines multiple agents with specific roles, goals, and backstories. Each agent is configured to use the LLM defined earlier.

In [6]:
senior_technical_writer = Agent(
    role="Senior Technical Writer",
    
    goal="""Craft clear, engaging, and well-structured
            technical content based on research findings""",
    
    backstory="""You are an experienced technical writer
                with expertise in simplifying complex
                concepts, structuring content for readability,
                and ensuring accuracy in documentation.""",
                
    llm=llm,
                
    verbose=True
)

research_analyst = Agent(
    role="Senior Research Analyst",
    goal="""Find, analyze, and summarize information 
            from various sources to support technical 
            and business-related inquiries.""",
    backstory="""You are a skilled research analyst with expertise 
                in gathering accurate data, identifying key trends, 
                and presenting insights in a structured manner.""",
    llm=llm,
    verbose=True
)

code_reviewer = Agent(
    role="Senior Code Reviewer",
    goal="""Review code for bugs, inefficiencies, and 
            security vulnerabilities while ensuring adherence 
            to best coding practices.""",
    backstory="""You are a seasoned software engineer with years of 
                experience in writing, reviewing, and optimizing 
                production-level code in multiple programming languages.""",
    llm=llm,
    verbose=True
)

legal_reviewer = Agent(
    role="Legal Document Expert Reviewer",
    goal="""Review contracts and legal documents to 
            ensure compliance with applicable laws and 
            highlight potential risks.""",
    backstory="""You are a legal expert with deep knowledge 
                of contract law, regulatory frameworks, 
                and risk mitigation strategies.""",
    llm=llm,
    verbose=True
)


## Define a Writing Task

This block defines a writing task for the Senior Technical Writer agent. The task includes a description, the agent responsible, and the expected output.

In [7]:

writing_task = Task(
    description="""Write a well-structured, engaging,
                   and technically accurate article
                   on {topic}.""",
    
    agent=senior_technical_writer, 
    
    
    expected_output="""A polished, detailed, and easy-to-read
                       article on the given topic.""",
)


## Create a Crew and Execute the Task

This block creates a crew to manage the task and agents. It then kicks off the crew with the specified input and displays the response.

In [8]:
crew = Crew(
    agents=[senior_technical_writer],
    tasks=[writing_task],
    verbose=True
)

response = crew.kickoff(inputs={"topic":"AI Agents"})

# Display the response in Markdown format
Markdown(response.raw)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 96fe9383-a845-41e0-9597-855d2776dc1e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Technical Writer                                                                                 │
│                                                                                                                 │
│  Task: Write a well-structured, engaging,                                                                       │
│                     and technically accurate article                                                            │
│                     on AI Agents.                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

20:00:48 - LiteLLM:DEBUG: utils.py:338 - 

20:00:48 - LiteLLM:DEBUG: utils.py:338 - Request to litellm:
20:00:48 - LiteLLM:DEBUG: utils.py:338 - litellm.completion(model='ollama/gemma3n:latest', messages=[{'role': 'system', 'content': 'You are Senior Technical Writer. You are an experienced technical writer\n                with expertise in simplifying complex\n                concepts, structuring content for readability,\n                and ensuring accuracy in documentation.\nYour personal goal is: Craft clear, engaging, and well-structured\n            technical content based on research findings\nTo give my best complete final answer to the task respond using the exact following format:\n\nThought: I now can give a great answer\nFinal Answer: Your final answer must be the great and the most complete as possible, it must be outcome described.\n\nI MUST use these formats, my job depends on it!'}, {'role': 'user', 'content': '\nCurrent Task: Write a well-structured, engaging,\n    

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Technical Writer                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## AI Agents: The Future of Task Automation and Intelligent Assistance                                         │
│                                                                                                                 │
│  **Introduction:**                                                                                              │
│                                                                                                                 │
│  Artificial Intelligence (AI) is rapidly evolving, moving beyond narrow applications to encompass more          │
│  autonomous and intelligent systems.  A key development in this field is the rise of **AI Agents**.  These      │
│  agents represent a significant leap forward in automation, offering the potential to perform complex tasks,    │
│  make decisions, and interact with the world in a way that mimics human cognitive abilities. This article will  │
│  explore what AI agents are, how they work, their different types, potential applications, challenges, and the  │
│  future they promise.                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  **What are AI Agents?**                                                                                        │
│                                                                                                                 │
│  At its core, an AI agent is an autonomous entity that perceives its environment through sensors and acts upon  │
│  that environment through actuators. Think of it as a software entity designed to:                              │
│                                                                                                                 │
│  *   **Perceive:** Gather information about its surroundings using data inputs (e.g., text, images, sensor      │
│  data).                                                                                                         │
│  *   **Reason:** Analyze the perceived information to understand the current situation and determine the best   │
│  course of action.                                                                                              │
│  *   **Act:** Execute actions in the environment to achieve specific goals.                                     │
│  *   **Learn:** Adapt its behavior based on experience to improve its performance over time.                    │
│                                                                                                                 │
│  Unlike traditional software that requires explicit instructions for every possible scenario, AI agents can     │
│  leverage machine learning to generalize from past experiences and handle novel situations. They are designed   │
│  to be proactive, not just reactive, and can pursue goals with varying degrees of autonomy.                     │
│                                                                                                                 │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ed7a66b3-a57e-4365-807d-348d44a534ac                                                                     │
│  Agent: Senior Technical Writer                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 96fe9383-a845-41e0-9597-855d2776dc1e                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ## AI Agents: The Future of Task Automation and Intelligent Assistance                           │
│                                                                                                                 │
│  **Introduction:**                                                                                              │
│                                                                                                                 │
│  Artificial Intelligence (AI) is rapidly evolving, moving beyond narrow applications to encompass more          │
│  autonomous and intelligent systems.  A key development in this field is the rise of **AI Agents**.  These      │
│  agents represent a significant leap forward in automation, offering the potential to perform complex tasks,    │
│  make decisions, and interact with the world in a way that mimics human cognitive abilities. This article will  │
│  explore what AI agents are, how they work, their different types, potential applications, challenges, and the  │
│  future they promise.                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  **What are AI Agents?**                                                                                        │
│                                                                                                                 │
│  At its core, an AI agent is an autonomous entity that perceives its environment through sensors and acts upon  │
│  that environment through actuators. Think of it as a software entity designed to:                              │
│                                                                                                                 │
│  *   **Perceive:** Gather information about its surroundings using data inputs (e.g., text, images, sensor      │
│  data).                                                                                                         │
│  *   **Reason:** Analyze the perceived information to understand the current situation and determine the best   │
│  course of action.                                                                                              │
│  *   **Act:** Execute actions in the environment to achieve specific goals.                                     │
│  *   **Learn:** Adapt its behavior based on experience to improve its performance over time.                    │
│                                                                                                                 │
│  Unlike traditional software that requires explicit instructions for every possible scenario, AI agents can     │
│  leverage machine learning to generalize from past experiences and handle novel situations. They are designed   │
│  to be proactive, not just reactive, and can pursue goals with varying degrees of autonomy.                     │
│                                                       

## AI Agents: The Future of Task Automation and Intelligent Assistance

**Introduction:**

Artificial Intelligence (AI) is rapidly evolving, moving beyond narrow applications to encompass more autonomous and intelligent systems.  A key development in this field is the rise of **AI Agents**.  These agents represent a significant leap forward in automation, offering the potential to perform complex tasks, make decisions, and interact with the world in a way that mimics human cognitive abilities. This article will explore what AI agents are, how they work, their different types, potential applications, challenges, and the future they promise.



**What are AI Agents?**

At its core, an AI agent is an autonomous entity that perceives its environment through sensors and acts upon that environment through actuators. Think of it as a software entity designed to:

*   **Perceive:** Gather information about its surroundings using data inputs (e.g., text, images, sensor data).
*   **Reason:** Analyze the perceived information to understand the current situation and determine the best course of action.
*   **Act:** Execute actions in the environment to achieve specific goals.
*   **Learn:** Adapt its behavior based on experience to improve its performance over time.

Unlike traditional software that requires explicit instructions for every possible scenario, AI agents can leverage machine learning to generalize from past experiences and handle novel situations. They are designed to be proactive, not just reactive, and can pursue goals with varying degrees of autonomy.



**How do AI Agents Work?**

The functionality of an AI agent typically relies on a combination of several key components:

1.  **Perception:** This involves using sensors (e.g., APIs, web scraping tools, natural language processing models) to gather data from the environment.  For example, an agent designed to monitor social media might use APIs to collect tweets, posts, and comments.

2.  **Decision-Making:** This is the "brain" of the agent. It uses algorithms and models to analyze the perceived data, plan actions, and select the most appropriate course of action. Common techniques include:

    *   **Reinforcement Learning (RL):** The agent learns through trial and error, receiving rewards or penalties for its actions.  This is particularly useful for tasks where the optimal strategy is not immediately obvious.
    *   **Planning Algorithms:**  These algorithms break down complex tasks into smaller, manageable steps.  They can be used to create a sequence of actions that will achieve a desired outcome.
    *   **Knowledge Representation:**  Agents often need to store and reason about knowledge about the world. This can be done using knowledge graphs, ontologies, or other structured representations.

3.  **Action:**  This involves using actuators (e.g., APIs, command-line tools, web browsers) to execute the chosen actions in the environment.  For instance, an agent might use an API to send an email, update a database, or control a physical robot.

4.  **Memory:**  Agents need to remember past experiences to learn and improve. This can be achieved using various memory structures, such as short-term memory (for recent events) and long-term memory (for general knowledge).



**Types of AI Agents:**

AI agents can be categorized based on their complexity and capabilities:

*   **Simple Reflex Agents:** These agents react directly to sensory inputs based on pre-defined rules. They have no memory of past events.  (e.g., a thermostat that turns on the heater when the temperature drops below a certain threshold).

*   **Model-Based Reflex Agents:**  These agents maintain an internal model of the world, allowing them to reason about the consequences of their actions.

*   **Goal-Based Agents:** These agents have explicit goals and use planning algorithms to achieve them. They can reason about the best sequence of actions to reach their goals.

*   **Utility-Based Agents:**  These agents not only consider whether they achieve their goals but also evaluate the desirability of different outcomes. They aim to maximize their "utility" – a measure of how good an outcome is.

*   **Learning Agents:**  These agents can learn from experience and improve their performance over time. They use machine learning techniques to update their models and policies.



**Applications of AI Agents:**

The potential applications of AI agents are vast and span numerous industries:

*   **Customer Service:**  AI-powered chatbots can handle routine customer inquiries, freeing up human agents to focus on more complex issues.  Advanced agents can personalize interactions and proactively offer assistance.
*   **Financial Trading:**  Algorithmic trading agents can analyze market data and execute trades automatically, potentially generating higher returns.
*   **Healthcare:**  AI agents can assist with diagnosis, treatment planning, and patient monitoring. They can analyze medical images, predict patient outcomes, and personalize care plans.
*   **Supply Chain Management:**  Agents can optimize logistics, predict demand, and manage inventory levels.
*   **Content Creation:**  AI agents can generate text, images, and videos, assisting with marketing, journalism, and entertainment.
*   **Software Development:**  AI agents can automate code generation, testing, and debugging.
*   **Robotics:**  AI agents are the brains behind autonomous robots, enabling them to perform complex tasks in unstructured environments (e.g., warehouse automation, delivery robots).
*   **Personal Assistants:**  More sophisticated personal assistants can manage schedules, make travel arrangements, and proactively provide information based on user preferences.



**Challenges and Considerations:**

Despite their potential, AI agents face several challenges:

*   **Safety and Reliability:** Ensuring that AI agents operate safely and reliably is paramount.  Unforeseen behaviors can have serious consequences, especially in critical applications.
*   **Explainability and Transparency:**  Understanding *why* an AI agent makes a particular decision is crucial for building trust and accountability.  "Black box" AI agents can be difficult to debug and validate.
*   **Bias:** AI agents can inherit biases from the data they are trained on, leading to unfair or discriminatory outcomes.
*   **Security:** AI agents can be vulnerable to attacks, such as adversarial examples, which can cause them to malfunction.
*   **Ethical Considerations:**  The use of AI agents raises ethical questions about job displacement, privacy, and autonomy.
*   **Data Requirements:** Training effective AI agents often requires large amounts of high-quality data.
*   **Computational Resources:**  Developing and deploying AI agents can be computationally expensive.



**The Future of AI Agents:**

The future of AI agents is bright.  We can expect to see:

*   **Increased Autonomy:** Agents will become more capable of operating independently, with less human intervention.
*   **Improved Reasoning Abilities:**  Advances in areas like neuro-symbolic AI will enable agents to reason more effectively and handle more complex tasks.
*   **Greater Adaptability:**  Agents will be able to adapt to changing environments and learn from new experiences more quickly.
*   **More Natural Interaction:**  Agents will be able to interact with humans in more natural and intuitive ways, using voice, gestures, and other modalities.
*   **Specialized Agents:**  We will see the development of highly specialized agents designed for specific tasks and industries.
*   **Integration with the Metaverse:** AI agents will play a key role in the development of the metaverse, enabling more immersive and interactive experiences.



**Conclusion:**

AI agents represent a transformative technology with the potential to revolutionize the way we work and live.  While challenges remain, the ongoing advancements in AI are paving the way for a future where intelligent agents can automate complex tasks, augment human capabilities, and solve some of the world's most pressing problems.  As AI agents become more sophisticated, it's crucial to address the ethical and societal implications to ensure that they are used for the benefit of humanity.

## Save the Response to a File

This block saves the raw response from the crew execution to a Markdown file for further use or documentation.

In [9]:
f = open("build/output.md", "w")
f.write(response.raw)
f.close()

# Define a file reader tool (OPENAI ONLY - DOES NOT WORK WITH OLLAMA MODELS)

In [ ]:
from crewai_tools import FileReadTool

file_read_tool = FileReadTool()

## Troubleshooting Note

**Issue:** The original code was encountering an `IndexError` in the litellm/ollama integration when using CrewAI tools. The error occurred in the prompt template processing where `messages[msg_i]` was accessing an index out of range.

**Root Cause:** This appears to be a compatibility issue between:
- CrewAI's tool integration system
- LiteLLM's Ollama prompt template processing
- The FileReadTool parameter validation

**Solution:** 
1. **Enabled debugging** with `litellm._turn_on_debug()` to get detailed error information
2. **Removed the FileReadTool** from the agent to avoid the tool integration bug
3. **Read file content directly** in Python and embedded it in the task description
4. **Simplified the workflow** to avoid dynamic parameter passing

This approach maintains the same functionality while working around the integration issue.

**Results:**
- This solution did not work either.  I was unable to get CrewAI FileReadTool or any custom specialized Tool Function to work with ollama based GenAI models.  The tool usage only worked for OpenAI for some reason.

## Define the Agent For Reading And Summarizing Files (OPENAI ONLY)

In [ ]:
from crewai import Agent

summarizer_agent = Agent(
    role="Senior Document Summarizer",

    goal="Extract and summarize key insights from provided files in 20 words or less.",
    backstory="""You are an expert in document analysis, skilled at extracting 
                 key details, summarizing content, and identifying critical 
                 insights from structured and unstructured text.""",
    llm=llm,
    tools=[file_read_tool],
    verbose=True
)

## Create Reading and Analyzing File Task (OPENAI ONLY)

In [ ]:
from crewai import Task

summarizer_task = Task(
    description=(
        "Use the FileReadTool to read the contents of {file_path}"
        "and provide a summary in 20 words or less. "
        "Ensure the summary captures the key insights "
        "and main points from the document."
    ),
    agent=summarizer_agent,
    tools=[file_read_tool],
    expected_output="A concise 20-word summary of the key points from the file.",
)

## Assemble A File-Processing Crew Workflow (OPENAI ONLY)

In [ ]:
from crewai import Crew

summarizer_crew = Crew(
    agents=[summarizer_agent],
    tasks=[summarizer_task],
    verbose=True
)

# Run the crew with the file input
result = summarizer_crew.kickoff(inputs={"file_path": "build/output.md"})

In [ ]:
# Display the response in Markdown format
from IPython.display import Markdown
Markdown(result.raw)

# Building multi-agent systems

## Define our Serper Dev tool

The Serper Dev Tool is a utility designed to interact with search engines and retrieve relevant information programmatically. It is particularly useful for tasks that require real-time data or insights from the web, such as:

1. Conducting research on specific topics.
2. Gathering up-to-date information for decision-making.
3. Enhancing the capabilities of agents by providing them with access to external data sources.

### How It Can Be Used
- **Integration with Agents**: The tool can be integrated into agents to enable them to fetch and process web-based information dynamically.
- **Custom Queries**: Developers can define specific queries to retrieve targeted information, making it adaptable to various use cases.
- **Real-Time Insights**: By leveraging the tool, agents can provide real-time insights and context, improving the quality and relevance of their outputs.

This tool is ideal for scenarios where static data is insufficient, and dynamic, real-time information is required to achieve the desired outcomes.

In [15]:
from crewai_tools import SerperDevTool

serper_dev_tool = SerperDevTool()

/home/vscode/.local/lib/python3.11/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


## Define the Internet Researcher Agent and Task

This block defines an `Internet Researcher` agent and a corresponding research task. 
- The agent is equipped with the `SerperDevTool` to perform web-based research. 
- The agent's role is to find the most relevant and recent information about a given topic, leveraging its expertise in navigating the internet and gathering reliable data. 
- The task specifies the use of the `SerperDevTool` to extract key insights from multiple sources and produce a detailed research report with references.

In [16]:
from crewai import Agent, Task

research_agent = Agent(
    role="Internet Researcher",
    goal="Find the most relevant and recent information about a given topic.",
    backstory="""You are a skilled researcher, adept at navigating the internet 
                 and gathering high-quality, reliable information.""",
    tools=[serper_dev_tool],
    verbose=True
)

research_task = Task(
    description="""Use the SerperDevTool to search for the 
                   most relevant and recent data about {topic}."""
                "Extract the key insights from multiple sources.",
    agent=research_agent,
    tools=[serper_dev_tool],
    expected_output="A detailed research report with key insights and source references."
)

## Define The Summarization Agent

This agent is responsible for condensing the research into a concise and structured summary. The Summarization Agent ensures that the research findings are structured, easy to read, and clear.

In [17]:
summarizer_agent = Agent(
    role="Content Summarizer",
    goal="Condense the key insights from research into a short and informative summary.",
    backstory="""You are an expert in distilling complex information into concise, 
                 easy-to-read summaries.""",
    verbose=True
)

summarization_task = Task(
    description="Summarize the research report into a concise and informative paragraph. "
                "Ensure clarity, coherence, and completeness.",
    agent=summarizer_agent,
    expected_output="A well-structured summary with the most important insights."
)

## Define The Fact-Checking Agent

The Fact-Checking Agent will cross-check all summarized information with credible sources:

- The Fact-Checking Agent is responsible for validating the summarized information.
- The Serper Dev Tool is used again to cross-check facts with external sources.

In [18]:
fact_checker_agent = Agent(
    role="Fact-Checking Specialist",
    goal="Verify the accuracy of information and remove any misleading or false claims.",
    backstory="""You are an investigative journalist with a knack for validating facts, 
                 ensuring that only accurate information is published.""",
    tools=[serper_dev_tool],
    verbose=True
)

fact_checking_task = Task(
    description="Verify the summarized information for accuracy using the SerperDevTool. "
                "Cross-check facts with reliable sources and correct any errors.",
    agent=fact_checker_agent,
    tools=[serper_dev_tool],
    expected_output="A fact-checked, verified summary of the research topic."
)

## Create Multi-Agent Crew Workflow

- All three agents are grouped into a Crew, each assigned their specific task.
- Tasks are executed sequentially in a structured workflow:  Research → Summarization → Fact-Checking.
- The topic is dynamically provided at runtime, making the workflow flexible for any research topic.

In [19]:
from crewai import Crew, Process

research_crew = Crew(
    agents=[research_agent, summarizer_agent, fact_checker_agent],
    tasks=[research_task, summarization_task, fact_checking_task],
    process=Process.sequential,
    verbose=True
)


## Kickoff The Multi-Agent Crew Workflow

In [ ]:

result = research_crew.kickoff(inputs={"topic": "The impact of AI on job markets"})

# print("\nFinal Verified Summary:\n", result)
Markdown(result.raw)

# Using YAML Based Agent and Workflow Definition

## Load YAML File

In [21]:
import yaml

with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)
    
# Uncomment the following line to print the configuration
# 
# from pprint import pprint
# pprint(config)

## Convert Agent Definitions to use YAML Config Details

In [22]:
research_agent = Agent(
    role=config["agents"]["research_agent"]["role"],
    goal=config["agents"]["research_agent"]["goal"],
    backstory=config["agents"]["research_agent"]["backstory"],
    tools=[serper_dev_tool],
    verbose=True
)

research_task = Task(
    description=config["tasks"]["research_task"]["description"],
    agent=research_agent,
    tools=[serper_dev_tool],
    expected_output=config["tasks"]["research_task"]["expected_output"]
)

summarization_agent = Agent(
    role=config["agents"]["summarization_agent"]["role"],
    goal=config["agents"]["summarization_agent"]["goal"],
    backstory=config["agents"]["summarization_agent"]["backstory"],
    verbose=True
)

fact_checker_agent = Agent(
    role=config["agents"]["fact_checker_agent"]["role"],
    goal=config["agents"]["fact_checker_agent"]["goal"],
    backstory=config["agents"]["fact_checker_agent"]["backstory"],
    tools=[serper_dev_tool],
    verbose=True
)

summarization_task = Task(
    description=config["tasks"]["summarization_task"]["description"],
    agent=summarization_agent,
    expected_output=config["tasks"]["summarization_task"]["expected_output"],
)

fact_checking_task = Task(
    description=config["tasks"]["fact_checking_task"]["description"],
    agent=fact_checker_agent,
    tools=[serper_dev_tool],
    expected_output=config["tasks"]["fact_checking_task"]["expected_output"],
)


## Create Multi-Agent Crew Workflow

- All three agents are grouped into a Crew, each assigned their specific task.
- Tasks are executed sequentially in a structured workflow:  Research → Summarization → Fact-Checking.
- The topic is dynamically provided at runtime, making the workflow flexible for any research topic.

In [ ]:
from crewai import Crew, Process

research_crew = Crew(
    agents=[research_agent, summarizer_agent, fact_checker_agent],
    tasks=[research_task, summarization_task, fact_checking_task],
    process=Process.sequential,
    verbose=True
)

result = research_crew.kickoff(inputs={"topic": "The impact of AI on job markets"})
print("\nFinal Verified Summary:\n", result)